In [11]:
import os
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np 
import shapely 

import osm_flex.download as dl
import osm_flex.extract as ex
from osm_flex.simplify import remove_contained_points,remove_exact_duplicates
from osm_flex.config import OSM_DATA_DIR,DICT_GEOFABRIK

from tqdm import tqdm

from lonboard import viz
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Blues_9

from pathlib import Path
import pathlib

In [12]:
#define paths
p = Path('..')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'

In [13]:
def country_download(iso3):
    dl.get_country_geofabrik(iso3)
    data_loc = OSM_DATA_DIR.joinpath(f'{DICT_GEOFABRIK[iso3][1]}-latest.osm.pbf')
    return data_loc

def overlay_hazard_assets(df_ds,assets):
    """
    Overlay hazard assets on a dataframe of spatial geometries.
    Arguments:
        *df_ds*: GeoDataFrame containing the spatial geometries of the hazard data. 
        *assets*: GeoDataFrame containing the infrastructure assets.
    Returns:
        *geopandas.GeoSeries*: A GeoSeries containing the spatial geometries of df_ds that intersect with the infrastructure assets.
    """
    #overlay 
    hazard_tree = shapely.STRtree(df_ds.geometry.values)
    if (shapely.get_type_id(assets.iloc[0].geometry) == 3) | (shapely.get_type_id(assets.iloc[0].geometry) == 6): # id types 3 and 6 stand for polygon and multipolygon
        return  hazard_tree.query(assets.geometry,predicate='intersects')    
    else:
        return  hazard_tree.query(assets.buffered,predicate='intersects')

def buffer_assets(assets,buffer_size=0.00083):
    """
    Buffer spatial assets in a GeoDataFrame.
    Arguments:
        *assets*: GeoDataFrame containing spatial geometries to be buffered.
        *buffer_size* (float, optional): The distance by which to buffer the geometries. Default is 0.00083.
    Returns:
        *GeoDataFrame*: A new GeoDataFrame with an additional 'buffered' column containing the buffered geometries.
    """
    assets['buffered'] = shapely.buffer(assets.geometry.values,distance=buffer_size)
    return assets

def get_damage_per_asset(asset,hazard_numpified,asset_geom,hazard_intensity,fragility_values,maxdam_asset):
    """
    Calculate damage for a given asset based on hazard information.
    Arguments:
        *asset*: Tuple containing information about the asset. It includes:
            - Index or identifier of the asset (asset[0]).
            - Asset-specific information, including hazard points (asset[1]['hazard_point']).
        *flood_numpified*: NumPy array representing flood hazard information.
        *asset_geom*: Shapely geometry representing the spatial coordinates of the asset.
        *curve*: Pandas DataFrame representing the curve for the asset type.
        *maxdam*: Maximum damage value.
    Returns:
        *tuple*: A tuple containing the asset index or identifier and the calculated damage.
    """
    
    # find the exact hazard overlays:
    get_hazard_points = hazard_numpified[asset[1]['hazard_point'].values] 
    get_hazard_points[shapely.intersects(get_hazard_points[:,1],asset_geom)]

    # estimate damage
    if len(get_hazard_points) == 0: # no overlay of asset with hazard
        return 0
    
    else:
        if asset_geom.geom_type == 'LineString':
            overlay_meters = shapely.length(shapely.intersection(get_hazard_points[:,1],asset_geom)) # get the length of exposed meters per hazard cell
            return np.sum((np.interp(np.float16(get_hazard_points[:,0]),hazard_intensity,fragility_values))*overlay_meters*maxdam_asset) #return asset number, total damage for asset number (damage factor * meters * max. damage)
        elif asset_geom.geom_type in ['MultiPolygon','Polygon']:
            overlay_m2 = shapely.area(shapely.intersection(get_hazard_points[:,1],asset_geom))
            return np.sum((np.interp(np.float16(get_hazard_points[:,0]),hazard_intensity,fragility_values))*overlay_m2*maxdam_asset)
        elif asset_geom.geom_type == 'Point':
            return np.sum((np.interp(np.float16(get_hazard_points[:,0]),hazard_intensity,fragility_values))*maxdam_asset)

def read_hazard_data(data_path,hazard_type):

    if hazard_type == 'fluvial':
        hazard_data = data_path / 'Floods' / 'Jamaica' / 'fluvial_undefended' # need to make country an input
        return list(hazard_data.iterdir())

    elif hazard_type == 'pluvial':
        hazard_data = data_path / 'Floods' / 'Jamaica' / 'pluvial' # need to make country an input
        return list(hazard_data.iterdir())

    elif hazard_type == 'windstorm':
        hazard_data = data_path / 'Windstorms' 
        return list(hazard_data.iterdir())

    elif hazard_type == 'earthquake':
        hazard_data = data_path / 'Earthquakes' 
        return list(hazard_data.iterdir())

    elif hazard_type == 'landslides':
        hazard_data = data_path / 'Landslides' 
        return list(hazard_data.iterdir())


def read_vul_maxdam(data_path,hazard_type,infra_type):

    vul_data = data_path / 'Vulnerability'

    if hazard_type in ['pluvial','fluvial']:  
        curves = pd.read_excel(vul_data / 'Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx',sheet_name = 'F_Vuln_Depth',index_col=[0],header=[0,1,2,3,4])
    elif hazard_type == 'windstorm':
        curves = pd.read_excel(vul_data / 'Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx',sheet_name = 'W_Vuln_V10m',index_col=[0],header=[0,1,2,3,4])

    infra_curves =  curves.loc[:, curves.columns.get_level_values('Infrastructure description').str.lower().str.contains(infra_type)]
    
    maxdam = pd.read_excel(vul_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
    infra_maxdam = maxdam.loc[maxdam.index.get_level_values('Infrastructure description').str.lower().str.contains(infra_type),'Amount'].dropna()
    infra_maxdam = infra_maxdam[pd.to_numeric(infra_maxdam, errors='coerce').notnull()]

    return infra_curves,infra_maxdam

def read_flood_map(flood_map_path):
    flood_map = xr.open_dataset(flood_map_path, engine="rasterio")

    flood_map_vector = flood_map['band_data'].to_dataframe().reset_index() #transform to dataframe
    
    #remove data that will not be used
    flood_map_vector = flood_map_vector.loc[(flood_map_vector.band_data > 0) & (flood_map_vector.band_data < 100)]
    
    # create geometry values and drop lat lon columns
    flood_map_vector['geometry'] = [shapely.points(x) for x in list(zip(flood_map_vector['x'],flood_map_vector['y']))]
    flood_map_vector = flood_map_vector.drop(['x','y','band','spatial_ref'],axis=1)
    
    # drop all non values to reduce size
    flood_map_vector = flood_map_vector.loc[~flood_map_vector['band_data'].isna()].reset_index(drop=True)
    
    # and turn them into squares again:
    flood_map_vector.geometry= shapely.buffer(flood_map_vector.geometry,distance=0.00083/2,cap_style='square').values # distance should be made an input still!

    return flood_map_vector

def read_windstorm_map(windstorm_map_path,bbox):
     
    # load data from NetCDF file
    with xr.open_dataset(flood_map_path) as ds:
        
        # convert data to WGS84 CRS
        ds.rio.write_crs(4326, inplace=True)
        ds = ds.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])
        #ds['band_data'] = ds['band_data']/0.88*1.11 #convert 10-min sustained wind speed to 3-s gust wind speed
    
        ds_vector = ds['band_data'].to_dataframe().reset_index() #transform to dataframe
        
        #remove data that will not be used
        ds_vector = ds_vector.loc[(ds_vector.band_data > 0) & (ds_vector.band_data < 100)]
        
        # create geometry values and drop lat lon columns
        ds_vector['geometry'] = [shapely.points(x) for x in list(zip(ds_vector['x'],ds_vector['y']))]
        ds_vector = ds_vector.drop(['x','y','band','spatial_ref'],axis=1)
        ds_vector['geometry'] = shapely.buffer(ds_vector.geometry, distance=0.1/2, cap_style='square').values
    
        return ds_vector

In [41]:
def country_infrastructure_hazard(data_path,country_code,infra_type,hazard_type):
    
    # get country osm data
    data_loc = country_download(country_code)
    
    # get infrastructure data:
    assets = ex.extract_cis(data_loc, infra_type)
    
    # convert assets to epsg3857
    assets = gpd.GeoDataFrame(assets).set_crs(4326).to_crs(3857)
    
    if infra_type == 'road':
        assets = assets.loc[assets.geometry.geom_type == 'LineString']
        assets = assets.rename(columns={'highway' : 'asset'})
    elif infra_type == 'rail':
        assets = assets.loc[assets.geometry.geom_type == 'LineString']
        assets = assets.rename(columns={'railway' : 'asset'})
    elif infra_type == 'education':
        
        assets = assets.rename(columns={'building' : 'asset'})
        assets = assets.reset_index(drop=True)
        assets = remove_contained_points(assets)

        #convert points to polygons
        assets.loc[assets.geom_type == 'Point','geometry'] = assets.loc[assets.geom_type == 'Point'].buffer(
                                                                        distance=np.sqrt(assets.loc[assets.geom_type == 'MultiPolygon'].area.median())/2, cap_style='square')

    elif infra_type == 'air':
        assets = assets.rename(columns={'aeroway' : 'asset'})
        
    
    # create dicts for quicker lookup
    geom_dict = assets['geometry'].to_dict()
    type_dict = assets['asset'].to_dict()
    
    # read hazard data
    hazard_data_list = read_hazard_data(data_path,hazard_type)
    
    # read vulnerability and maxdam data:
    infra_curves,maxdams = read_vul_maxdam(data_path,hazard_type,infra_type)
    
    # start analysis 
    print(f'{country_code} runs for {infra_type} for {hazard_type} for {len(hazard_data_list)} maps for {len(infra_curves.T)*len(maxdams)} combinations')
    
    if hazard_type in ['windstorm','earthquake','landslide']:
        # load country geometry file and create geometry to clip
        ne_countries = gpd.read_file(data_path / "natural_earth" / "ne_10m_admin_0_countries.shp")
        bbox = ne_countries.loc[ne_countries['ISO_A3']==country_code].geometry.envelope.values[0].bounds
        
    collect_output = {}
    for single_footprint in hazard_data_list: #tqdm(hazard_data_list,total=len(hazard_data_list)):
    
        hazard_name = single_footprint.parts[-1].split('.')[0]
        
        # load hazard map
        if hazard_type in ['pluvial','fluvial']:
            hazard_map = read_flood_map(single_footprint)
        elif hazard_type in ['windstorm']:
             hazard_map = read_windstorm_map(single_footprint,bbox)
        elif hazard_type in ['earthquake']:
             hazard_map = read_earthquake_map(single_footprint)
        elif hazard_type in ['landslide']:
             hazard_map = read_landslide_map(single_footprint)
         
        # convert hazard data to epsg 3857
        hazard_map = gpd.GeoDataFrame(hazard_map).set_crs(4326).to_crs(3857)
                  
        # overlay assets:
        overlay_assets = pd.DataFrame(overlay_hazard_assets(hazard_map,buffer_assets(assets)).T,columns=['asset','hazard_point'])
    
        # convert dataframe to numpy array
        hazard_numpified = hazard_map.to_numpy() 

        curve_types = {'primary': ['F7.1', 'F7.2'],
                      'secondary': ['F7.3', 'F7.4']}

        maxdam_types = {}
        
        for infra_curve in infra_curves:
            # get curves
            curve = infra_curves[infra_curve[0]]
            hazard_intensity = curve.index.values
            fragility_values = (np.nan_to_num(curve.values,nan=(np.nanmax(curve.values)))).flatten()
                    
            for maxdam in maxdams:

                collect_inb = {}
                for asset in tqdm(overlay_assets.groupby('asset'),total=len(overlay_assets.asset.unique())): #group asset items for different hazard points per asset and get total number of unique assets
                    asset_type = type_dict[asset[0]]
                    
                    # if not curve in curve_types[asset_type]: 
                    #     collect_inb[asset_type] = 0
                        
                    # if np.max(fragility_values) == 0:
                    #     collect_inb[asset_type] = 0  
                    # else:
                    asset_geom = geom_dict[asset[0]]
                                
                    collect_inb[asset[0]] = asset_type,get_damage_per_asset(asset,hazard_numpified,asset_geom,hazard_intensity,fragility_values,maxdam)
                collect_output[hazard_name,infra_curve[0],maxdam] = pd.DataFrame(pd.DataFrame(collect_inb).T.values,columns=['asset_type','damage']).groupby('asset_type').sum()
        break # remove break after testing
    return collect_output

In [19]:
# List of critical infrastructure systems to process
cis = ['healthcare', 'education', 'gas', 'oil', 'telecom', 'water', 'wastewater', 'power', 'rail', 'road', 'air']

In [20]:
hazard_type='pluvial'
infra_type='road'
country_code='JAM'

In [21]:
infra_curves,maxdams = read_vul_maxdam(data_path,hazard_type,infra_type)

In [22]:
for infra_curve in infra_curves:
    # get curves
    curve = infra_curves[infra_curve[0]]

In [23]:
infra_curves.columns.get_level_values(0)

Index(['F7.1', 'F7.2', 'F7.2a (lower boundary)', 'F7.2b (upper boundary)',
       'F7.3', 'F7.4', 'F7.5', 'F7.6', 'F7.7', 'F7.8', 'F7.9', 'F7.10',
       'F7.11', 'F7.12', 'F7.13'],
      dtype='object', name='ID number')

In [42]:
test = country_infrastructure_hazard(data_path,country_code,infra_type,hazard_type)

extract points: 0it [00:00, ?it/s]
extract lines: 100%|██████████████████████████████████████████████████████████| 39952/39952 [00:03<00:00, 10452.58it/s]


JAM runs for road for pluvial for 10 maps for 255 combinations


100%|████████████████████████████████████████████████████████████████████████████| 8539/8539 [00:02<00:00, 3599.92it/s]


In [1]:
curves

NameError: name 'curves' is not defined